In [36]:
import pandas as pd
from scipy import stats
from scipy.stats import mode
salesdata = pd.read_csv('/home/prashant/Desktop/competation/bigmartsales.csv')


In [5]:
salesdata.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [37]:
cat_dict ={}

unique_outlet_size_val = salesdata.Outlet_Size.dropna().unique()
copy_outlet_size = salesdata.Outlet_Size.dropna()
 
count =0
for val in unique_outlet_size_val:
    cat_dict[val] =int(count)
    count =count+1
salesdata['Outlet_Size'] =copy_outlet_size.apply(lambda x :cat_dict[x])
xx =salesdata.pivot_table(columns ='Outlet_Type',values = 'Outlet_Size',aggfunc = (lambda x : (mode(x).mode[0]))).astype(int) 
#print(xx)

miss_val = salesdata.Outlet_Size.isnull()
#print(miss_val)

salesdata.loc[miss_val,'Outlet_Size'] = salesdata.loc[miss_val,'Outlet_Type'].apply(lambda x : xx[x])
salesdata.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64